In [1]:
pip install python-terrier==0.12.1 nltk scikit-learn lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sebim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = pt.get_dataset("irds:nfcorpus")

In [4]:
from pathlib import Path

index = pt.index.IterDictIndexer(
    str(Path.cwd()),  # this will be ignored
    meta={
        "docno": 16,
        "title": 256,
        "abstract": 65536,
        "url": 128,
    },
    type=pt.index.IndexingType.MEMORY,
).index(dataset.get_corpus_iter(), fields=["title", "abstract", "url"])

Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
nfcorpus documents: 100%|██████████| 5371/5371 [00:04<00:00, 1171.81it/s]


In [5]:
base_model = pt.terrier.FeaturesRetriever(
    index,
    wmodel="BM25",
    features=["WMODEL:BM25", "WMODEL:PL2", "WMODEL:DPH"],
    num_results=100,
    metadata=["docno", "title", "abstract", "url"],
)

In [6]:
from models.lambdaMART import *
from models.lambdaRank import *
from models.rankSVM import *

models = [
    ("lambdaMART", get_lambdaMART_model(base_model)),
    ("lambdaRank", get_lambdaRank_model(base_model)),
    ("rankSVM", get_rankSVM_model(base_model)),
]


fitting_args = (
    pt.get_dataset("irds:nfcorpus/train/nontopic").get_topics(),
    pt.get_dataset("irds:nfcorpus/train/nontopic").get_qrels(),
    pt.get_dataset("irds:nfcorpus/dev/nontopic").get_topics(),
    pt.get_dataset("irds:nfcorpus/dev/nontopic").get_qrels(),
)

for model_name, model in models:
    print(f"Training {model_name}")
    model.fit(*fitting_args)


Training lambdaMART
18:19:56.753 [main] WARN org.terrier.querying.ApplyTermPipeline -- The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001225 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 109480, number of used features: 3
Training lambdaRank
[LightGBM] [Warning] Unknown parameter: index
[LightGBM] [Warning] Unknown parameter: index
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 109480, number of used features: 3
[LightGBM

In [7]:
from pyterrier.measures import nDCG, RR, MAP

basic_evaluations = pt.Experiment(
    [base_model] + [model for _, model in models],
    pt.get_dataset("irds:nfcorpus/test/nontopic").get_topics(),
    pt.get_dataset("irds:nfcorpus/test/nontopic").get_qrels(),
    names=["BM25"] + [model_name for model_name, _ in models],
    eval_metrics=[nDCG @ 10, RR @ 10, MAP],
)

basic_evaluations

c:\Projects\information_retrieval_ltr_comparison\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
c:\Projects\information_retrieval_ltr_comparison\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: index


,name,nDCG@10,RR@10,AP
0,BM25,0.243556,0.451312,0.092002
1,lambdaMART,0.245393,0.440647,0.095133
2,lambdaRank,0.245815,0.447826,0.095976
3,rankSVM,0.122083,0.258116,0.045800


In [8]:
from fairness import fairness_evaluation, compute_df

baseline_df = compute_df(
    base_model, 
    pt.get_dataset("irds:nfcorpus/test/nontopic").get_topics(),
    pt.get_dataset("irds:nfcorpus/test/nontopic").get_qrels(),
)

model_dfs = [
    compute_df(
        model, 
        pt.get_dataset("irds:nfcorpus/test/nontopic").get_topics(), 
        pt.get_dataset("irds:nfcorpus/test/nontopic").get_qrels()
    ) for _, model in models
]

print("Baseline")
display(baseline_df)
print("Models")
for model_name, model_df in zip([model_name for model_name, _ in models], model_dfs):
    print(model_name)
    display(model_df)


c:\Projects\information_retrieval_ltr_comparison\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
c:\Projects\information_retrieval_ltr_comparison\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: index
Baseline


,qid,query,docid,rank,features,docno,title,abstract,url,score,label
8,PLAIN-102,stopping heart disease in childhood,3252,8,"[13.348406938450136, 6.970303613119249, 9.4059...",MED-3253,Pathobiological determinants of atherosclerosi...,Abstract OBJECTIVES: Atherosclerosis begins in...,http://www.ncbi.nlm.nih.gov/pubmed/17015535,13.348407,3.0
101,PLAIN-112,food dyes and adhd,3379,1,"[34.1572336095778, 15.98155624248299, 17.97454...",MED-3380,Artificial food dyes and attention deficit hyp...,Abstract Attention deficit hyperactivity disor...,http://www.ncbi.nlm.nih.gov/pubmed/21729092,34.157234,3.0
105,PLAIN-112,food dyes and adhd,3381,5,"[18.276464524453594, 8.032497120445777, 9.2371...",MED-3382,Dietary sensitivities and ADHD symptoms: thirt...,Abstract Artificial food colors (AFCs) have no...,http://www.ncbi.nlm.nih.gov/pubmed/21127082,18.276465,3.0
178,PLAIN-112,food dyes and adhd,3374,78,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3375,"Classroom ""cupcake"" celebrations: observations...",Abstract OBJECTIVE: To describe food and bever...,http://www.ncbi.nlm.nih.gov/pubmed/22032916,2.407661,2.0
179,PLAIN-112,food dyes and adhd,3375,79,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3376,"""Split them!"" smaller item sizes of cookies le...",Abstract OBJECTIVE: Examine the influence of a...,http://www.ncbi.nlm.nih.gov/pubmed/22391143,2.407661,2.0
...,...,...,...,...,...,...,...,...,...,...,...
13765,PLAIN-91,chronic headaches and pork parasites,3169,34,"[10.037061317130387, 4.5865208815502605, 5.821...",MED-3170,Cognitive Changes and Quality of Life in Neuro...,Abstract Background Few studies have focused o...,http://www.ncbi.nlm.nih.gov/pubmed/22303492,10.037061,2.0
13796,PLAIN-91,chronic headaches and pork parasites,3175,65,"[8.557335409722581, 3.7420062171766797, 4.0928...",MED-3176,Is dementia reversible in patients with neuroc...,Abstract Methods: Ninety consecutive patients ...,http://www.ncbi.nlm.nih.gov/pubmed/16024900,8.557335,3.0
13808,PLAIN-91,chronic headaches and pork parasites,4954,77,"[8.207073616642692, 3.567083088594999, 4.01155...",MED-4955,Toxoplasma and schizophrenia. - PubMed - NCBI,Abstract Research on infectious agents as a po...,http://www.ncbi.nlm.nih.gov/pubmed/19825110,8.207074,2.0
13813,PLAIN-91,chronic headaches and pork parasites,3306,82,"[8.137466361371592, 3.508564251191885, 4.56298...",MED-3307,Mortality in workers employed in pig abattoirs...,Abstract OBJECTIVE: workers in slaughterhouses...,http://www.ncbi.nlm.nih.gov/pubmed/21724184,8.137466,2.0


Models
lambdaMART


,qid,query,docid,features,docno,title,abstract,url,score,rank,label
8,PLAIN-102,stopping heart disease in childhood,3252,"[13.348406938450136, 6.970303613119249, 9.4059...",MED-3253,Pathobiological determinants of atherosclerosi...,Abstract OBJECTIVES: Atherosclerosis begins in...,http://www.ncbi.nlm.nih.gov/pubmed/17015535,-0.673081,10,3.0
101,PLAIN-112,food dyes and adhd,3379,"[34.1572336095778, 15.98155624248299, 17.97454...",MED-3380,Artificial food dyes and attention deficit hyp...,Abstract Attention deficit hyperactivity disor...,http://www.ncbi.nlm.nih.gov/pubmed/21729092,2.434518,1,3.0
105,PLAIN-112,food dyes and adhd,3381,"[18.276464524453594, 8.032497120445777, 9.2371...",MED-3382,Dietary sensitivities and ADHD symptoms: thirt...,Abstract Artificial food colors (AFCs) have no...,http://www.ncbi.nlm.nih.gov/pubmed/21127082,0.631453,5,3.0
178,PLAIN-112,food dyes and adhd,3374,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3375,"Classroom ""cupcake"" celebrations: observations...",Abstract OBJECTIVE: To describe food and bever...,http://www.ncbi.nlm.nih.gov/pubmed/22032916,-4.009069,71,2.0
179,PLAIN-112,food dyes and adhd,3375,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3376,"""Split them!"" smaller item sizes of cookies le...",Abstract OBJECTIVE: Examine the influence of a...,http://www.ncbi.nlm.nih.gov/pubmed/22391143,-4.009069,72,2.0
...,...,...,...,...,...,...,...,...,...,...,...
13765,PLAIN-91,chronic headaches and pork parasites,3169,"[10.037061317130387, 4.5865208815502605, 5.821...",MED-3170,Cognitive Changes and Quality of Life in Neuro...,Abstract Background Few studies have focused o...,http://www.ncbi.nlm.nih.gov/pubmed/22303492,-1.332080,45,2.0
13796,PLAIN-91,chronic headaches and pork parasites,3175,"[8.557335409722581, 3.7420062171766797, 4.0928...",MED-3176,Is dementia reversible in patients with neuroc...,Abstract Methods: Ninety consecutive patients ...,http://www.ncbi.nlm.nih.gov/pubmed/16024900,-2.445378,98,3.0
13808,PLAIN-91,chronic headaches and pork parasites,4954,"[8.207073616642692, 3.567083088594999, 4.01155...",MED-4955,Toxoplasma and schizophrenia. - PubMed - NCBI,Abstract Research on infectious agents as a po...,http://www.ncbi.nlm.nih.gov/pubmed/19825110,-2.328638,95,2.0
13813,PLAIN-91,chronic headaches and pork parasites,3306,"[8.137466361371592, 3.508564251191885, 4.56298...",MED-3307,Mortality in workers employed in pig abattoirs...,Abstract OBJECTIVE: workers in slaughterhouses...,http://www.ncbi.nlm.nih.gov/pubmed/21724184,-2.014467,83,2.0


lambdaRank


,qid,query,docid,features,docno,title,abstract,url,score,rank,label
8,PLAIN-102,stopping heart disease in childhood,3252,"[13.348406938450136, 6.970303613119249, 9.4059...",MED-3253,Pathobiological determinants of atherosclerosi...,Abstract OBJECTIVES: Atherosclerosis begins in...,http://www.ncbi.nlm.nih.gov/pubmed/17015535,-0.408397,8,3.0
101,PLAIN-112,food dyes and adhd,3379,"[34.1572336095778, 15.98155624248299, 17.97454...",MED-3380,Artificial food dyes and attention deficit hyp...,Abstract Attention deficit hyperactivity disor...,http://www.ncbi.nlm.nih.gov/pubmed/21729092,1.908611,1,3.0
105,PLAIN-112,food dyes and adhd,3381,"[18.276464524453594, 8.032497120445777, 9.2371...",MED-3382,Dietary sensitivities and ADHD symptoms: thirt...,Abstract Artificial food colors (AFCs) have no...,http://www.ncbi.nlm.nih.gov/pubmed/21127082,0.591824,8,3.0
178,PLAIN-112,food dyes and adhd,3374,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3375,"Classroom ""cupcake"" celebrations: observations...",Abstract OBJECTIVE: To describe food and bever...,http://www.ncbi.nlm.nih.gov/pubmed/22032916,-2.607161,78,2.0
179,PLAIN-112,food dyes and adhd,3375,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3376,"""Split them!"" smaller item sizes of cookies le...",Abstract OBJECTIVE: Examine the influence of a...,http://www.ncbi.nlm.nih.gov/pubmed/22391143,-2.607161,79,2.0
...,...,...,...,...,...,...,...,...,...,...,...
13765,PLAIN-91,chronic headaches and pork parasites,3169,"[10.037061317130387, 4.5865208815502605, 5.821...",MED-3170,Cognitive Changes and Quality of Life in Neuro...,Abstract Background Few studies have focused o...,http://www.ncbi.nlm.nih.gov/pubmed/22303492,-1.049866,48,2.0
13796,PLAIN-91,chronic headaches and pork parasites,3175,"[8.557335409722581, 3.7420062171766797, 4.0928...",MED-3176,Is dementia reversible in patients with neuroc...,Abstract Methods: Ninety consecutive patients ...,http://www.ncbi.nlm.nih.gov/pubmed/16024900,-1.874868,94,3.0
13808,PLAIN-91,chronic headaches and pork parasites,4954,"[8.207073616642692, 3.567083088594999, 4.01155...",MED-4955,Toxoplasma and schizophrenia. - PubMed - NCBI,Abstract Research on infectious agents as a po...,http://www.ncbi.nlm.nih.gov/pubmed/19825110,-1.883967,99,2.0
13813,PLAIN-91,chronic headaches and pork parasites,3306,"[8.137466361371592, 3.508564251191885, 4.56298...",MED-3307,Mortality in workers employed in pig abattoirs...,Abstract OBJECTIVE: workers in slaughterhouses...,http://www.ncbi.nlm.nih.gov/pubmed/21724184,-1.489806,60,2.0


rankSVM


,qid,query,docid,features,docno,title,abstract,url,score,rank,label
8,PLAIN-102,stopping heart disease in childhood,3252,"[13.348406938450136, 6.970303613119249, 9.4059...",MED-3253,Pathobiological determinants of atherosclerosi...,Abstract OBJECTIVES: Atherosclerosis begins in...,http://www.ncbi.nlm.nih.gov/pubmed/17015535,0.099850,96,3.0
101,PLAIN-112,food dyes and adhd,3379,"[34.1572336095778, 15.98155624248299, 17.97454...",MED-3380,Artificial food dyes and attention deficit hyp...,Abstract Attention deficit hyperactivity disor...,http://www.ncbi.nlm.nih.gov/pubmed/21729092,1.461047,1,3.0
105,PLAIN-112,food dyes and adhd,3381,"[18.276464524453594, 8.032497120445777, 9.2371...",MED-3382,Dietary sensitivities and ADHD symptoms: thirt...,Abstract Artificial food colors (AFCs) have no...,http://www.ncbi.nlm.nih.gov/pubmed/21127082,0.099905,15,3.0
178,PLAIN-112,food dyes and adhd,3374,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3375,"Classroom ""cupcake"" celebrations: observations...",Abstract OBJECTIVE: To describe food and bever...,http://www.ncbi.nlm.nih.gov/pubmed/22032916,0.097797,68,2.0
179,PLAIN-112,food dyes and adhd,3375,"[2.4076614941037597, 2.1696885625573916, 3.043...",MED-3376,"""Split them!"" smaller item sizes of cookies le...",Abstract OBJECTIVE: Examine the influence of a...,http://www.ncbi.nlm.nih.gov/pubmed/22391143,0.097797,69,2.0
...,...,...,...,...,...,...,...,...,...,...,...
13765,PLAIN-91,chronic headaches and pork parasites,3169,"[10.037061317130387, 4.5865208815502605, 5.821...",MED-3170,Cognitive Changes and Quality of Life in Neuro...,Abstract Background Few studies have focused o...,http://www.ncbi.nlm.nih.gov/pubmed/22303492,0.100002,64,2.0
13796,PLAIN-91,chronic headaches and pork parasites,3175,"[8.557335409722581, 3.7420062171766797, 4.0928...",MED-3176,Is dementia reversible in patients with neuroc...,Abstract Methods: Ninety consecutive patients ...,http://www.ncbi.nlm.nih.gov/pubmed/16024900,0.100051,46,3.0
13808,PLAIN-91,chronic headaches and pork parasites,4954,"[8.207073616642692, 3.567083088594999, 4.01155...",MED-4955,Toxoplasma and schizophrenia. - PubMed - NCBI,Abstract Research on infectious agents as a po...,http://www.ncbi.nlm.nih.gov/pubmed/19825110,0.100047,48,2.0
13813,PLAIN-91,chronic headaches and pork parasites,3306,"[8.137466361371592, 3.508564251191885, 4.56298...",MED-3307,Mortality in workers employed in pig abattoirs...,Abstract OBJECTIVE: workers in slaughterhouses...,http://www.ncbi.nlm.nih.gov/pubmed/21724184,0.100167,4,2.0


In [10]:
fairness_evaluations = [fairness_evaluation(model_df, baseline_df) for model_df in model_dfs]

for model_name, fairness_evaluation in zip([model_name for model_name, _ in models], fairness_evaluations):
    print(model_name)
    display(fairness_evaluation)

lambdaMART


{'InterQuery': {'mean_nDCG': np.float64(0.9368),
  'std_nDCG': np.float64(0.0955),
  'range_nDCG': np.float64(0.4969),
  'fairness_score': np.float64(0.898)},
 'LabelInversionRate': np.float64(0.159),
 'IndividualFairnessViolation': np.float64(0.0),
 'KendallsTauVsBaseline': np.float64(0.6942)}

lambdaRank


{'InterQuery': {'mean_nDCG': np.float64(0.9399),
  'std_nDCG': np.float64(0.0939),
  'range_nDCG': np.float64(0.4736),
  'fairness_score': np.float64(0.9001)},
 'LabelInversionRate': np.float64(0.1434),
 'IndividualFairnessViolation': np.float64(0.0),
 'KendallsTauVsBaseline': np.float64(0.8015)}

rankSVM


{'InterQuery': {'mean_nDCG': np.float64(0.9255),
  'std_nDCG': np.float64(0.1035),
  'range_nDCG': np.float64(0.4114),
  'fairness_score': np.float64(0.8882)},
 'LabelInversionRate': np.float64(0.1977),
 'IndividualFairnessViolation': np.float64(0.0),
 'KendallsTauVsBaseline': np.float64(0.0182)}